<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Data-Acquisition" data-toc-modified-id="Data-Acquisition-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Acquisition</a></span></li><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Preprocessing</a></span></li><li><span><a href="#Split-Training-&amp;-Test-Data" data-toc-modified-id="Split-Training-&amp;-Test-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Split Training &amp; Test Data</a></span></li><li><span><a href="#Create-Data-Visualization-Method" data-toc-modified-id="Create-Data-Visualization-Method-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create Data Visualization Method</a></span></li><li><span><a href="#Select-Training-Data-Subset" data-toc-modified-id="Select-Training-Data-Subset-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Select Training Data Subset</a></span></li><li><span><a href="#Visualize-Data" data-toc-modified-id="Visualize-Data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Visualize Data</a></span></li><li><span><a href="#Create-CNN-Model-Method" data-toc-modified-id="Create-CNN-Model-Method-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Create CNN Model Method</a></span></li><li><span><a href="#Instantiate-Model" data-toc-modified-id="Instantiate-Model-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Instantiate Model</a></span></li><li><span><a href="#Set-Training-Hyperparameters" data-toc-modified-id="Set-Training-Hyperparameters-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Set Training Hyperparameters</a></span></li><li><span><a href="#Create-Callback-Settings" data-toc-modified-id="Create-Callback-Settings-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Create Callback Settings</a></span></li><li><span><a href="#Train-Model" data-toc-modified-id="Train-Model-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Train Model</a></span></li></ul></div>

#### Import Libraries

In [1]:
import glob
import numpy as np
import os.path as path
import imageio
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import CSVLogger
from sklearn.metrics import classification_report
from datetime import datetime

Using TensorFlow backend.


#### Data Acquisition

In [ ]:
'https://www.kaggle.com/rhammell/planesnet'

'https://www.kaggle.com/rhammell/planesnet'

#### Data Preprocessing

In [ ]:
# Set the path to planesnet image folder.
IMAGE_PATH = 'C:/Users/596596/Documents/GitHub/UVA-DataPalooza/planesnet/'
file_paths = glob.glob(path.join(IMAGE_PATH, '*.png'))

# Load the images into a single variable and convert to a numpy array.
images = [imageio.imread(path) for path in file_paths]
images = np.asarray(images)

# Get the dimensions of the image.  This image_size array will let us know the dimensions of the images. If we print this 
# variable, we learn that the dimensions of the images are [20, 20, 3]. This means that each image in the dataset has 20 
# rows, 20 columns, and a depth of 3 (or 3 channels, Red, Green, and Blue). These numbers define the spatial resolution of 
# the image.
image_size = np.asarray([images.shape[1], images.shape[2], images.shape[3]])
print(image_size)

# Scale the image.  BThe images we are working with are 8-bit. This means that each pixel in the image is one of 256 (2⁸ = 256)
# possible values. While some machine learning algorithms can handle having relatively large pixel values, most methods perform
# optimally (train within our lifetimes) when small, floating point values are processed. Hence we divide by 255 
# (0 is a possible pixel value as well, so 255 is actually the maximum value found within our data) to scale our data between 
# 0–1. So, after this step, a value that was represented by an integer of 128, would now instead be represented by a floating 
# point value of 128/255 = ~0.502. It is important to note that the appearance of the images will be fundamentally unaltered 
# after this step.
images = images / 255

# Read the labels from the filenames.  These labels were manually annotated by a human being. Each 20 by 20 image was reviewed, 
# and given a “1” (True) if it contained an airplane and a “0” (False) if it did not contain an airplane. We can extract these 
# labels by reading in the first character in the image filename. For example, in the image filename 
# “0__20140723_181317_0905__-122.14328662_37.697282118.png”, the very first “0” is indicating to us that this image does not 
# contain an airplane.
n_images = images.shape[0]
labels = np.zeros(n_images)
for i in range(n_images):
    filename = path.basename(file_paths[i])[0]
    labels[i] = int(filename[0])
    
# Now we images and their corresponding labels. In the classic supervised machine learning terminiology, we consider 
# the images to be “X” and the labels to be “y”. The machine learning algorithm is going to find a function “f” such that for
# any given image, y = f(X). In other words, we are going to take the 1,200 (20 x 20 x 3 = 1,200) X values, run them through a 
# function, f, and predict one singular value, y, that represents a classification of either “plane” or “not plane”.

#### Split Training & Test Data

In [ ]:
# Split into test and training sets.  This practice is important, because it is the only way to evaluate your model in 
# an unbiased way. Basically, you want your model to learn on the training set, (usually about 90% of all of the images you 
# have available), and then report back its accuracy by evaluating it on the test set (the remaining 10%).  Generally, a model 
# that only performs well on data that it was trained on is not useful or interesting, and is described as “over-fit”. 
# Over-fitting is the equivalent of the model memorizing the labels. It would be like a student who learns that 2 + 2 = 4, and 
# then assumes that every addition problem they see is equal to 4.  They have not actually learned how to perform addition, 
# rather they have just memorized an output. We want our model to generalize a concept. It needs to be able to grab an image 
# with an unknown label and accurately predict if it contains an airplane or not, regardless of whether or not the model has 
# seen the image before. We can best simulate this scenario by creating a test set, as shown below. 
TRAIN_TEST_SPLIT = 0.9

# Split at the given index
split_index = int(TRAIN_TEST_SPLIT * n_images)
shuffled_indices = np.random.permutation(n_images)
train_indices = shuffled_indices[0:split_index]
test_indices = shuffled_indices[split_index:]

# Split the images and the labels (Optimize, Fold)
x_train = images[train_indices, :, :]
y_train = labels[train_indices]
x_test = images[test_indices, :, :]
y_test = labels[test_indices]

#### Create Data Visualization Method

In [ ]:
# This function will plot positive and negative image examples for us. By “positive” we mean data with a label of “1” or “True”,
# and by “negative” we mean data with a label of “0” or “False”. We need negative examples (i.e. images that do not contain 
# airplanes), because otherwise the model would not have any reference point and likely assume that the mere presence of an 
# image is indicative that it contains an airplane. The inner workings of the visualize_data function create two rows of images 
# where some number of positive examples are on the top, and the same number of negative examples are on the bottom. The titles 
# above them indicate what their label is.
def visualize_data(positive_images, negative_images):
    figure = plt.figure(figsize=(15,5))
    count = 0
    for i in range(positive_images.shape[0]):
        count += 1
        figure.add_subplot(2, positive_images.shape[0], count)
        plt.imshow(positive_images[i, :, :])
        plt.axis('off')
        plt.title("1")

        figure.add_subplot(1, negative_images.shape[0], count)
        plt.imshow(negative_images[i, :, :])
        plt.axis('off')
        plt.title("0")
    plt.show()

#### Select Training Data Subset

In [ ]:
# Number of positive and negative examples to show
N_TO_VISUALIZE = 10

# Select the first N positive examples
positive_example_indices = (y_train == 1)
positive_examples = x_train[positive_example_indices, :, :]
positive_examples = positive_examples[0:N_TO_VISUALIZE, :, :]

# Select the first N negative examples
negative_example_indices = (y_train == 0)
negative_examples = x_train[negative_example_indices, :, :]
negative_examples = negative_examples[0:N_TO_VISUALIZE, :, :]

#### Visualize Data

In [ ]:
# The images only have a resolution of 20px by 20px and at this poor resolution we can somewhat pick out the airplanes in 
# the positive examples, and the lack of airplanes in the negative images. It seems we have our work cut out for us! That 
# being said, let’s list out some more detailed observations we might make from this data. There are partial airplanes in 
# the negative examples. The positive examples appear to contain airplanes of different sizes. Sometimes the plane occupies
# 16–18 pixels, other times only 8–10 pixels. There are varied atmospheric conditions present in the images. If you spend 
# more time looking at these examples, you will probably notice even more intricacies about the data. The devil is often 
# in the details, so it is worth your time to get to know your data as well as possible. 
# 1. There are partial airplanes in the negative examples
# 2. The positive examples appear to contain airplanes of different sizes. 
# 3. Sometimes the plane occupies 16–18 pixels, other times only 8–10 pixels
# 4. There are varied atmospheric conditions present in the images visualize_data(positive_examples, negative_examples)
visualize_data(positive_examples, negative_examples)

#### Create CNN Model Method

In [ ]:
def cnn(size):
    # INPUTS
    # size     - size of the input images
    # N_LAYERS - number of layers
    # OUTPUTS
    # model    - compiled CNN

    # Define hyperparamters
    N_LAYERS = 4
    MIN_NEURONS = 20
    MAX_NEURONS = 120
    KERNEL = (3, 3)

    # Determine the # of neurons in each convolutional layer
    steps = np.floor(MAX_NEURONS / (N_LAYERS + 1))
    nuerons = np.arange(MIN_NEURONS, MAX_NEURONS, steps)
    nuerons = nuerons.astype(np.int32)

    # Define a model
    model = Sequential()

    # Add convolutional layers
    for i in range(0, N_LAYERS):
        if i == 0:
            shape = (size[0], size[1], size[2])
            model.add(Conv2D(nuerons[i], KERNEL, input_shape=shape))
        else:
            model.add(Conv2D(nuerons[i], KERNEL))

        model.add(Activation('relu'))

    # Add max pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(MAX_NEURONS))
    model.add(Activation('relu'))

    # Add output layer
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # Print a summary of the model
    model.summary()

    return model

#### Instantiate Model

In [ ]:
model = cnn(size=image_size)

#### Set Training Hyperparameters

In [ ]:
EPOCHS = 1
BATCH_SIZE = 200

#### Create Callback Settings

In [ ]:
csv_logger = CSVLogger('log.csv', append=True, separator=',')

#### Train Model

In [ ]:
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data = (x_test, y_test), callbacks=[csv_logger])